In [1]:
import re
from urllib import request
import gzip
import shutil
import requests
import pandas as pd
import sqlalchemy
import urllib
from multiprocessing.pool import ThreadPool
import pyodbc

In [ ]:
def download_url(url):
    # Download process
    print("downloading: ",url)
    file_title = re.split(pattern='/', string=url)[-1]
    urlrtv = request.urlretrieve(url=url, filename=file_title)
    
    # for ".tsv" to ".csv"
    title = re.split(pattern=r'\.tsv', string=file_title)[0] +".csv"
    
    # Unzip ".gz" file
    with gzip.open(file_title, 'rb') as f_in:
        with open(title, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [ ]:
# URL List
urls = ["https://datasets.imdbws.com/title.episode.tsv.gz"
        ,"https://datasets.imdbws.com/title.ratings.tsv.gz"
        ,"https://datasets.imdbws.com/title.akas.tsv.gz"
        ,"https://datasets.imdbws.com/title.basics.tsv.gz"
        ,"https://datasets.imdbws.com/title.crew.tsv.gz"
        ,"https://datasets.imdbws.com/title.principals.tsv.gz"
        ,"https://datasets.imdbws.com/name.basics.tsv.gz"
        ]
# Run 5 multiple threads. Each call will take the next element in urls list
results = ThreadPool(5).imap_unordered(download_url, urls)
for r in results:
    print(r)

In [2]:
title_principals = pd.read_csv(r"E:\OneDrive - up.edu.mx\Up\Clases\Mineria de datos y CS\Clase 5\title.principals.csv", sep= "\t", nrows=1000)

In [3]:
title_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0005690,producer,producer,\N
3,tt0000001,4,nm0374658,cinematographer,director of photography,\N
4,tt0000002,1,nm0721526,director,\N,\N


In [4]:
from neo4j import GraphDatabase, basic_auth
import pyneoinstance

https://neo4j.com/developer-blog/scoobygraph-2/

In [9]:
# Python file - connection to Neo4j

# Retrieving the Neo4j connection credentials from the config.yaml file
configs=pyneoinstance.load_yaml_file('config.yaml')
creds=configs['credentials']

# Establishing the Neo4j connection
neo4j=pyneoinstance.Neo4jInstance(creds['uri'],creds['user'],creds['password'])

In [10]:
# Python - loading nodes
# Load series nodes
neo4j.execute_write_query_with_data(query =configs['cypher']['load_nodes']['load_series'],data=title_principals)
# Load episode nodes and IN_SERIES relationships
# neo4j.execute_write_query_with_data(query = configs['cypher']['load_nodes']['load_episodes'],data = title_principals)
# Load monster, subtype, type, and species nodes/relationships
# neo4j.execute_write_query_with_data(query = configs['cypher']['load_nodes']['load_monsters'],data = title_principals[~title_principals.monster_name.isna()])

e:\OneDrive - up.edu.mx\Up\Clases\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'labels_added': 461, 'nodes_created': 461, 'properties_set': 2766}

In [10]:
# Python script: loading main characters
# Loading the main characters
neo4j.execute_write_query(configs['cypher']['load_nodes']['load_main_characters'])

{'labels_added': 5, 'nodes_created': 5, 'properties_set': 5}

In [16]:
# Python - Creating APPEARS_WITH relationships
neo4j.execute_write_query(configs['cypher']['load_relationships']['APPEARS_WITH'])

{'relationships_created': 2260}

In [12]:
driver = GraphDatabase.driver(creds['uri'], auth=basic_auth(creds['user'], creds['password']))

In [24]:
cypher_query = '''
MATCH (n)
RETURN n AS count
LIMIT $limit
'''

In [25]:

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      limit=10).data())
  for record in results:
    print(record['count'])

driver.close()


C:\Users\halo4\AppData\Local\Temp\ipykernel_25876\212345830.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database="neo4j") as session:
C:\Users\halo4\AppData\Local\Temp\ipykernel_25876\212345830.py:2: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


{'index_': 1, 'titleID': 'tt0000001', 'personID': 'nm1588970', 'job_category': 'self', 'job_title': '\\N', 'characters_name': '["Self"]'}
{'index_': 1, 'titleID': 'tt0000002', 'personID': 'nm0721526', 'job_category': 'director', 'job_title': '\\N', 'characters_name': '\\N'}
{'index_': 1, 'titleID': 'tt0000003', 'personID': 'nm0721526', 'job_category': 'director', 'job_title': '\\N', 'characters_name': '\\N'}
{'index_': 1, 'titleID': 'tt0000004', 'personID': 'nm0721526', 'job_category': 'director', 'job_title': '\\N', 'characters_name': '\\N'}
{'index_': 1, 'titleID': 'tt0000005', 'personID': 'nm0443482', 'job_category': 'actor', 'job_title': '\\N', 'characters_name': '["Blacksmith"]'}
{'index_': 1, 'titleID': 'tt0000007', 'personID': 'nm0179163', 'job_category': 'actor', 'job_title': '\\N', 'characters_name': '\\N'}
{'index_': 1, 'titleID': 'tt0000008', 'personID': 'nm0653028', 'job_category': 'actor', 'job_title': '\\N', 'characters_name': '["Sneezing Man"]'}
{'index_': 1, 'titleID': 